In [ ]:
# - Between-eyebrows
# - Chin
# - Forehead
# - Left-cheek
# - Left-eye
# - Mouth
# - Nose
# - Right-cheek
# - Right-eye

In [13]:
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import cv2
model = YOLO("../yolo/runs/detect/train7/weights/best.pt")
names=  model.names


In [47]:
def face_region_counts(image_path):
    results = model.predict(image_path,conf=0.8)
    new_dict= {x:[] for x in range(len(names))}
    boxes = results[0].boxes
    for box in boxes:
        conf = box.conf.cpu().detach().numpy().tolist()
        index=  box.cls.cpu().detach().numpy().tolist()[0]
        new_dict[index].append(box.xyxy.cpu().detach().numpy().tolist()[0])
        # new_dict[)
        # print(conf)
        # print([names[x] for x in box.cls.cpu().detach().numpy().tolist()])
    check=False
    remove_columns = []
    for new in new_dict:
        if len(new_dict[new])<2:
            remove_columns.append(new)
        else:
            check=True
    for remove in remove_columns:
        del(new_dict[remove])
    
    if not check:
        return False
    return new_dict

In [46]:
face_region_counts('./botox_with bunny after/image_1.jpg')

FileNotFoundError: ./botox_with bunny after/image_1.jpg does not exist

In [54]:

# 얼굴 인식 함수

# 얼굴 인식
def split_face(file_path):
    image = cv2.imread(file_path)
    h,w,c = image.shape
    check = face_region_counts(file_path)
    print(check)
    filename = file_path.split('\\')[-1]
    file_path= "_".join(file_path.split('\\')[0].split(' '))
    # Find the position of the border
    if not os.path.exists(os.path.join('./data',file_path)):
        os.mkdir(os.path.join('./data',file_path))
    if not os.path.exists(os.path.join('./data',file_path,'before')):
        os.mkdir(os.path.join('./data',file_path,'before'))
    if not os.path.exists(os.path.join('./data',file_path,'after')):
        os.mkdir(os.path.join('./data',file_path,'after'))
    
    top_image_path =  f'./data\\{file_path}\\before\\{filename}'
    bottom_image_path = f'./data\\{file_path}\\after\\{filename}'
    slice_horizon =  0 # 수평
    slice_verticle = 0  #수직 
    if not check:
        return None
    else:
        for region in check:
            # print("REGION",region)
            # print(slice_horizon,slice_verticle)
            checking_x_y = [0,0,0,0]
            faces =  check[region]  
            center_faces= []
            for face in faces:
                mid_x,mid_y = (face[0]+face[2])/2,(face[1]+face[3])/2
                center_faces.append([mid_x,mid_y])
            center_faces.sort()
            min_x,max_x = center_faces[0][0],center_faces[-1][0]
            center_faces.sort(key=lambda x:x[1])
            min_y,max_y = center_faces[0][1],center_faces[-1][1]
            # print(center_faces)
            # print(min_x,min_y,max_x,max_y)
            # print("x :",max_x-min_x,"y :",max_y-min_y)
            if max_x-min_x >max_y-min_y:
                if region==2 or region==6:
                    slice_verticle+=4
                else:
                    slice_verticle+=1
            else:
                if region==2 or region==6:
                    slice_horizon+=4
                else:   
                    slice_horizon+=1
    if slice_verticle>slice_horizon:
        # 양옆으로 자르기
        # half_width = ((prev_x2-prev_x1)+(x2-x1))//2
        half_width = w // 2
        # print('xdiff : ',x_diff)
        # print("half :",half_width)
        left_image = image[:, :half_width]
        right_image = image[:, half_width:]
        # 이미지 저장
        cv2.imwrite(top_image_path,left_image)
        cv2.imwrite(bottom_image_path,right_image)
    elif slice_horizon<slice_verticle:
        # 위아래로 자르기
        half_height = h//2
        # print('xdiff : ',x_diff)
        # print("half :",half_height)
        upper_face = image[:half_height,:]
        lower_face = image[half_height:,:]
        # 이미지 저장
        cv2.imwrite(top_image_path,upper_face)
        cv2.imwrite(bottom_image_path,lower_face)
        # print('save end')
    else:
        print('SAME')
   

In [55]:
# dataset\botox_befor
# e_and_after_with_forehead\train\botox before and after with forehead_0.png
from glob import glob
from tqdm.auto import  tqdm
image_path= glob("*/true/*.jpg",recursive=True)
# print(image_path)
image_path


['botox_with bunny line\\true\\image_0.jpg',
 'botox_with bunny line\\true\\image_1.jpg',
 'botox_with bunny line\\true\\image_101.jpg',
 'botox_with bunny line\\true\\image_103.jpg',
 'botox_with bunny line\\true\\image_105.jpg',
 'botox_with bunny line\\true\\image_106.jpg',
 'botox_with bunny line\\true\\image_107.jpg',
 'botox_with bunny line\\true\\image_110.jpg',
 'botox_with bunny line\\true\\image_111.jpg',
 'botox_with bunny line\\true\\image_113.jpg',
 'botox_with bunny line\\true\\image_117.jpg',
 'botox_with bunny line\\true\\image_118.jpg',
 'botox_with bunny line\\true\\image_119.jpg',
 'botox_with bunny line\\true\\image_12.jpg',
 'botox_with bunny line\\true\\image_122.jpg',
 'botox_with bunny line\\true\\image_126.jpg',
 'botox_with bunny line\\true\\image_128.jpg',
 'botox_with bunny line\\true\\image_129.jpg',
 'botox_with bunny line\\true\\image_130.jpg',
 'botox_with bunny line\\true\\image_131.jpg',
 'botox_with bunny line\\true\\image_132.jpg',
 'botox_with bunny

In [57]:
for pat in tqdm(image_path):
    try:
        split_face(pat)
        # print(a,b)
    except Exception as e:
    #     print(e)
        pass

  0%|          | 0/2767 [00:00<?, ?it/s]


image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_0.jpg: 224x640 1 Between-eyebrows, 1 Nose, 3 Right-cheeks, 3 Right-eyes, 19.1ms
Speed: 0.0ms preprocess, 19.1ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 640)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_1.jpg: 640x512 1 Between-eyebrows, 2 Foreheads, 2 Mouths, 1 Nose, 1 Right-eye, 20.9ms
Speed: 3.0ms preprocess, 20.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_101.jpg: 448x640 2 Foreheads, 2 Noses, 2 Right-cheeks, 2 Right-eyes, 17.3ms
Speed: 3.1ms preprocess, 17.3ms inference, 3.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_103.jpg: 640x640 1 Forehead, 2 Left-eyes, 1 Mouth, 1 Nose, 2 Right-eyes, 26.3ms
Speed: 4.0ms preprocess, 26.3ms

{7: [[237.5474853515625, 82.3136978149414, 307.8098449707031, 156.8632354736328], [385.7076721191406, 89.55545043945312, 470.1690979003906, 157.11361694335938], [148.66549682617188, 87.03662872314453, 232.5037078857422, 156.6707000732422]], 8: [[366.6839294433594, 13.688992500305176, 471.40155029296875, 79.8531265258789], [235.3547821044922, 11.443206787109375, 331.1214294433594, 76.87359619140625], [121.15144348144531, 17.2116756439209, 234.80125427246094, 80.42654418945312]]}
{2: [[566.5214233398438, 328.2068176269531, 1120.3760986328125, 596.483642578125], [12.473143577575684, 317.0335388183594, 545.0941162109375, 581.9902954101562]], 5: [[152.2926025390625, 936.8077392578125, 422.4195556640625, 1084.7342529296875], [720.5753173828125, 976.4777221679688, 977.6735229492188, 1122.2783203125]]}
{2: [[0.0, 2.5435256958007812, 233.866943359375, 166.5534210205078], [399.88800048828125, 2.3473358154296875, 667.64501953125, 164.49798583984375]], 6: [[0.0, 411.181884765625, 108.2239151000976

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_105.jpg: 640x640 2 Between-eyebrowss, 1 Forehead, 27.2ms
Speed: 5.9ms preprocess, 27.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_106.jpg: 640x640 1 Between-eyebrows, 2 Noses, 1 Right-cheek, 4 Right-eyes, 23.7ms
Speed: 7.6ms preprocess, 23.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_107.jpg: 640x640 (no detections), 25.3ms
Speed: 0.7ms preprocess, 25.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\botox_with bunny line\true\image_110.jpg: 448x640 2 Between-eyebrowss, 1 Chin, 2 Foreheads, 1 Mouth, 20.1ms
Speed: 0.9ms preprocess, 20.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

i

{0: [[462.7729187011719, 703.9136962890625, 604.4902954101562, 877.6666259765625], [442.5851745605469, 192.53164672851562, 584.8697509765625, 412.2945251464844]]}
SAME
{6: [[160.80458068847656, 511.09814453125, 321.6023864746094, 625.4105224609375], [145.9539031982422, 167.25009155273438, 322.7680969238281, 281.101318359375]], 8: [[56.40434646606445, 6.354935169219971, 208.37493896484375, 142.65370178222656], [64.12493133544922, 357.1466369628906, 213.78411865234375, 489.17755126953125], [281.09002685546875, 5.369465351104736, 646.573974609375, 165.30642700195312], [292.62811279296875, 358.29644775390625, 662.749267578125, 508.27197265625]]}
SAME
False
{0: [[222.9095001220703, 245.37808227539062, 284.0277099609375, 351.4817810058594], [733.4097900390625, 248.5556182861328, 788.3043823242188, 350.7814025878906]], 2: [[634.0724487304688, 121.93843078613281, 893.0238647460938, 244.2909698486328], [134.70982360839844, 128.4976806640625, 385.0821838378906, 241.9719696044922]]}
{1: [[330.787

...

{0: [[151.8959503173828, 528.7731323242188, 181.8321533203125, 604.2978515625], [461.1751708984375, 540.6259765625, 492.79998779296875, 616.9326171875]], 2: [[399.19110107421875, 450.08160400390625, 559.3616333007812, 533.8765258789062], [394.306640625, 66.54690551757812, 559.4849853515625, 153.3284454345703], [88.2213134765625, 70.39227294921875, 242.29153442382812, 152.1836395263672], [91.3663330078125, 451.00885009765625, 246.79254150390625, 524.0460815429688]]}
SAME
{1: [[122.60465240478516, 1509.5224609375, 676.3428955078125, 1735.933837890625], [1288.260498046875, 1482.99560546875, 1779.122314453125, 1745.1944580078125]], 5: [[130.6348876953125, 1182.466064453125, 622.4819946289062, 1487.3013916015625], [1264.0986328125, 1190.67236328125, 1743.31005859375, 1463.762939453125]], 6: [[163.70465087890625, 875.7883911132812, 547.6507568359375, 1165.2755126953125], [1289.0152587890625, 865.9911499023438, 1666.285400390625, 1139.6539306640625]]}



image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\radiofrequency microneedling\true\image_95.jpg: 288x640 2 Mouths, 12.2ms
Speed: 2.1ms preprocess, 12.2ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)


{5: [[2243.01416015625, 1352.302001953125, 3108.796875, 2139.493408203125], [6062.27783203125, 1372.2310791015625, 7072.767578125, 2150.01318359375]]}



image 1/1 c:\Users\kihoon\Desktop\AI\skin_care\dataset\radiofrequency microneedling\true\image_99.jpg: 640x640 1 Forehead, 1 Left-eye, 1 Mouth, 17.2ms
Speed: 6.6ms preprocess, 17.2ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)


False


## make csv to dataset

In [ ]:
from glob import glob 
checking = glob('./*/*/*/before/*.png')
label = checking[0].split('\\')[1]
label , checking[0]

In [ ]:
from glob import glob
globing = glob('./*before_and_after*')
globing = [x.split('\\')[1] for x in globing]
label2id, id2label = dict(), dict()
for i, label in enumerate(globing):
    label2id[label] = i
    id2label[i] = label

id2label[2]

In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
labels = []
images = []
for j in checking:
    label = label2id[j.split('\\')[1]]
    path=  j
    labels.append(label)
    images.append(path)

In [ ]:
labels,images

In [ ]:
import pandas as pd
a = pd.DataFrame()
a['label']=labels
a['image']=images

In [ ]:
a.to_csv('./dataset4.csv',index=False)